In all versions 5.x the narrative gets cleaned as before and products are cleaned three times. The first time half of the three most occuring entries in __Product__ are randomly removed. The second time the same happens with the six most occuring ones. Finally, the components that almost never occur are removed. This run uses 5 different principle components. 

__Initialing the data__

In [1]:
import pandas as pd

df_selected1 = pd.read_csv("../balanced_data/corpus_balanced_cleaned_lemmatized_both_scarce_elimination.csv", encoding="utf-8")

print("nulls in df_selected:", df_selected1["Consumer complaint narrative"].isnull().sum())
df_selected = df_selected1.dropna()

nulls in df_selected: 5


__Initializing the bag of words__

In [2]:
from sklearn.feature_extraction.text import CountVectorizer

# vect (bag of words)
count_vect = CountVectorizer(
    stop_words="english",
    #ngram_range=(1,2), # bigrammen
    min_df=2, # only keep words that appear twice
    max_df=0.5 # appears max in 50% of documents
)

X_train_counts = count_vect.fit_transform(df_selected["Consumer complaint narrative"])

In [3]:
from sklearn.decomposition import TruncatedSVD

tSVD = TruncatedSVD(n_components=5)

principal_components = tSVD.fit_transform(X_train_counts)
print(principal_components.shape)

(203869, 5)


In [4]:
print(principal_components)
principal_components_df = pd.DataFrame(principal_components)
print(principal_components_df)

[[ 4.18481166 -3.60215287  0.07864812  0.74368383  0.01109701]
 [ 3.99935188 -1.1631372   0.68663759 -0.62149946  0.28618582]
 [ 2.07097064  0.89935291  2.25004262 -0.97246547 -0.49530679]
 ...
 [ 3.4946031  -3.52442265 -0.55910145  0.10504256  0.80855957]
 [ 2.46637612 -2.01368106 -1.34635291 -0.01348345  0.91737232]
 [ 1.58335223 -1.22226617  0.03063566  0.73870888 -0.62937727]]
                0          1         2         3          4
0        4.184812  -3.602153  0.078648  0.743684   0.011097
1        3.999352  -1.163137  0.686638 -0.621499   0.286186
2        2.070971   0.899353  2.250043 -0.972465  -0.495307
3       24.403531 -15.836370  3.414995  8.228719 -10.625432
4        3.270028   0.836831  2.221791 -1.323542  -0.310381
...           ...        ...       ...       ...        ...
203864   0.721678  -1.076698 -0.302435  0.075636   0.222101
203865   1.488503  -1.414682 -0.482761 -1.216981   1.878449
203866   3.494603  -3.524423 -0.559101  0.105043   0.808560
203867   2.46637

__One hot encoding of extra columns__

In [5]:
#One hot encoding
issue_df = pd.Categorical(df_selected['Issue'])

df_dummies = pd.get_dummies(issue_df, prefix = 'issue')

#X_train_counts_df = pd.DataFrame(X_train_counts)
#print("DF conversion done")

print(principal_components_df.shape)
print(df_dummies.shape)
df_concat = pd.concat([principal_components_df, df_dummies], axis = 1)
print(df_concat.shape)

(203869, 5)
(203869, 149)
(203869, 154)


__Split into train and test__

In [6]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(df_concat, df_selected['Product'])
print(X_train.shape)
print(X_test.shape)
print(Y_train.shape)
print(Y_test.shape)

(152901, 154)
(50968, 154)
(152901,)
(50968,)


__Application of linear SVM__

In [7]:
from sklearn import svm
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report # do more stats

classifier = svm.LinearSVC()
classifier.fit(X_train, Y_train)

test_predictions = classifier.predict(X_test)
print(classification_report(test_predictions, Y_test))

C:\Users\l-Larsovic-l\anaconda3\lib\site-packages\sklearn\svm\_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


                                                                              precision    recall  f1-score   support

                                                     Bank account or service       1.00      1.00      1.00      3702
                                                 Checking or savings account       0.99      1.00      1.00      4620
                                                               Consumer Loan       0.97      0.78      0.86      2911
                                                                 Credit card       1.00      1.00      1.00      4754
                                                 Credit card or prepaid card       0.95      0.97      0.96      4584
                                                            Credit reporting       1.00      1.00      1.00      4765
Credit reporting, credit repair services, or other personal consumer reports       0.99      0.87      0.93      6154
                                                       